In [2]:
import keras_ocr
def detect_w_keras(image_path):
    """Function returns detected text from image"""
   
    # Initialize pipeline
    pipeline = keras_ocr.pipeline.Pipeline()
    # Read in image path
    read_image = keras_ocr.tools.read(image_path)
    # prediction_groups is a list of (word, box) tuples
    prediction_groups = pipeline.recognize([read_image])
    return prediction_groups

In [3]:
import math
def get_distance(predictions):
    """
    Function returns dictionary with (key,value):
        * text : detected text in image
        * center_x : center of bounding box (x)
        * center_y : center of bounding box (y)
        * distance_from_origin : hypotenuse
        * distance_y : distance between y and origin (0,0)
    """
    
    # Point of origin
    x0, y0 = 0, 0
    # Generate dictionary
    detections = []
    for group in predictions:
        # Get center point of bounding box
        top_left_x, top_left_y = group[1][0]
        bottom_right_x, bottom_right_y = group[1][1]
        center_x = (top_left_x + bottom_right_x) / 2
        center_y = (top_left_y + bottom_right_y) / 2
    # Use the Pythagorean Theorem to solve for distance from origin
    distance_from_origin = math.dist([x0,y0], [center_x, center_y])
    # Calculate difference between y and origin to get unique rows
    distance_y = center_y - y0
    # Append all results
    detections.append({
                        'text':group[0],
                        'center_x':center_x,
                        'center_y':center_y,
                        'distance_from_origin':distance_from_origin,
                        'distance_y':distance_y
                    })
    return detections

In [4]:
def distinguish_rows(lst, thresh=15):
    """Function to help distinguish unique rows"""
    
    sublists = [] 
    for i in range(0, len(lst)-1):
        if lst[i+1]['distance_y'] - lst[i]['distance_y'] <= thresh:
            if lst[i] not in sublists:
                sublists.append(lst[i])
            sublists.append(lst[i+1])
        else:
            yield sublists
            sublists = [lst[i+1]]
    yield sublists

In [5]:
def main(image_path, thresh = 15, order='yes'):
    """
    Function returns predictions in human readable order 
    from left to right & top to bottom
    """
    
    predictions = detect_w_keras(image_path)
    #print(predictions)
    predictions = get_distance(predictions)
    #print(predictions)
    predictions = list(distinguish_rows(predictions, thresh))
    #print(predictions)
    # Remove all empty rows
    predictions = list(filter(lambda x:x!=[], predictions))
    #print(predictions)
    # Order text detections in human readable format
    ordered_preds = []
    ylst = ['yes', 'y']
    for pr in predictions:
        if order in ylst: 
            row = sorted(pr, key=lambda x:x['distance_from_origin'])
            for each in row: 
                ordered_preds.append(each['text'])
    return ordered_preds

In [6]:
main(r'test.jpg')

Looking for C:\Users\pskir\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\pskir\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 13s 13s/step


ValueError: too many values to unpack (expected 2)

In [7]:
pred = detect_w_keras(r"test.jpg")

Looking for C:\Users\pskir\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\pskir\.keras-ocr\crnn_kurapan.h5
1/1 [==============================] - 3s 3s/step


In [28]:
pred = pred[0]
print(pred)


[('third', array([[635.7658 , 174.12341],
       [791.11804, 186.48099],
       [787.3993 , 233.23062],
       [632.04706, 220.87308]], dtype=float32)), ('2c', array([[148.06369, 179.21655],
       [212.31209, 181.51114],
       [210.86624, 221.99522],
       [146.61786, 219.70064]], dtype=float32)), ('street', array([[ 875.8906 ,  178.18573],
       [1089.1323 ,  192.72491],
       [1085.4788 ,  246.31104],
       [ 872.23706,  231.77185]], dtype=float32)), ('2', array([[300.  , 181.25],
       [352.5 , 181.25],
       [352.5 , 221.25],
       [300.  , 221.25]], dtype=float32)), ('221', array([[421.25, 181.25],
       [523.75, 181.25],
       [523.75, 223.75],
       [421.25, 223.75]], dtype=float32)), ('ts', array([[243.75, 188.75],
       [307.5 , 188.75],
       [307.5 , 220.  ],
       [243.75, 220.  ]], dtype=float32)), ('s', array([[356.25, 196.25],
       [381.25, 196.25],
       [381.25, 215.  ],
       [356.25, 215.  ]], dtype=float32)), ('chandra', array([[138.07173, 315.171

In [29]:
import numpy as np

box_group = np.array([x[1] for x in pred])

In [30]:
box_group

array([[[ 635.7658 ,  174.12341],
        [ 791.11804,  186.48099],
        [ 787.3993 ,  233.23062],
        [ 632.04706,  220.87308]],

       [[ 148.06369,  179.21655],
        [ 212.31209,  181.51114],
        [ 210.86624,  221.99522],
        [ 146.61786,  219.70064]],

       [[ 875.8906 ,  178.18573],
        [1089.1323 ,  192.72491],
        [1085.4788 ,  246.31104],
        [ 872.23706,  231.77185]],

       [[ 300.     ,  181.25   ],
        [ 352.5    ,  181.25   ],
        [ 352.5    ,  221.25   ],
        [ 300.     ,  221.25   ]],

       [[ 421.25   ,  181.25   ],
        [ 523.75   ,  181.25   ],
        [ 523.75   ,  223.75   ],
        [ 421.25   ,  223.75   ]],

       [[ 243.75   ,  188.75   ],
        [ 307.5    ,  188.75   ],
        [ 307.5    ,  220.     ],
        [ 243.75   ,  220.     ]],

       [[ 356.25   ,  196.25   ],
        [ 381.25   ,  196.25   ],
        [ 381.25   ,  215.     ],
        [ 356.25   ,  215.     ]],

       [[ 138.07173,  315.17154],


In [31]:
def isOnSameLine(boxOne, boxTwo):
    boxOneStartY = boxOne[0,1]
    boxOneEndY = boxOne[2,1]
    boxTwoStartY = boxTwo[0,1]
    boxTwoEndY = boxTwo[2,1]
    if((boxTwoStartY <= boxOneEndY and boxTwoStartY >= boxOneStartY)
    or(boxTwoEndY <= boxOneEndY and boxTwoEndY >= boxOneStartY)
    or(boxTwoEndY >= boxOneEndY and boxTwoStartY <= boxOneStartY)):
        return True
    else:
        return False

def segmentLines(box_group):
    import numpy as np
    # sort by highest starty value (bottom left corner of box - [startX,startY], [endX,startY], [endX,endY], [startX, endY])
    box_group = box_group[np.argsort(box_group[:, 0, 1])]

    lined_box_group = np.zeros(box_group.shape)
    sorted_box_group = np.zeros(box_group.shape)

    # list of indexes
    temp = []
    i = 0
    lines_list = []

    # check if there is more than one box in the box_group
    if len(box_group) > 1:
        while i < len(box_group):
            for j in range(i + 1, len(box_group)):
                if(isOnSameLine(box_group[i],box_group[j])):
                    # print(str(i) + " and " + str(j) + " on same line")
                    if i not in temp:
                        temp.append(i)
                    if j not in temp:
                        temp.append(j)
                # else:
                    # print(str(i) + " and " + str(j) + " not on same line")
            # append temp with i if the current box (i) is not on the same line with any other box
            if len(temp) == 0:
                temp.append(i)
            
            # put boxes on same line into lined_box_group array
            lined_box_group = box_group[np.array(temp)]
            # sort boxes by startX value
            t = np.argsort(lined_box_group[:, 0, 0])
            lines_list.append([x+i for x in t])
            lined_box_group = lined_box_group[t]
            # copy sorted boxes on same line into sorted_box_group
            sorted_box_group[i:temp[-1]+1] = lined_box_group
            
            # skip to the index of the box that is not on the same line
            #print(t) 
            i = temp[-1] + 1
            # clear list of indexes 
            temp = []
    else:
        # since there is only one box in the boxgroup do nothing but copying the box
        # print("only one box in boxgroup")
        sorted_box_group = box_group
  
    return sorted_box_group,lines_list

In [32]:
s,l = segmentLines(box_group)
print(l)

[[2, 5, 3, 6, 4, 0, 1], [9, 7, 8], [10, 11, 12], [13, 14]]


In [33]:
for i in range(len(l)):
    l[i].sort()

print(l)


[[0, 1, 2, 3, 4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14]]


In [34]:
for ran in l:
    s_sub = s[ran[0]:ran[-1]+1]
    #print(s_sub)
    
    for x in s_sub:
        
        for text,box in pred:

            # print(round(x[0][0],1),round(box[0][0],1))
            # print(round(x[0][1],1), round(box[0][1],1))
            # print()
            # print( int(x[0][0]), int(x[0][1]) )
            # print( int(box[0][0]), int(box[0][1]) )
            # print()

            if int(x[0][0]) == int(box[0][0]) and int(x[0][1]) == int(box[0][1]):
                print(text)
                break

2c
ts
2
s
221
third
street
chandra
mooe
nagar
vedalapalems
n
elore
s21
oot


In [ ]:
# for ran in l:
#     temp_pred = pred[ran[0]:ran[-1]+1]
#     print(temp_pred)  
# for x in s:
#         for text,box in temp_pred:
#             #print(round(x[0][0],1),round(box[0][0],1))
#             #print(x[0][0],box[0][0])
#             if str(round(box[0][0],1)) == str(round(x[0][0],1)):
#                 print(text)
#                 break